In [ ]:
"""Iteration over outgoing edges, instead of nodes, see
https://nographs.readthedocs.io/en/latest/
graphs_and_adaptation.html#graphs-with-attributes"""
from functools import wraps
from decorator import decorator
import nographs as nog
from graphviz import Digraph

In [ ]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _edge, _trav ):
        dot.node( str(_edge), str(_edge) )
        for result in wrapped(_edge, _trav):
            edge = dot.edge( str(_edge), str(result[0]), str(result[1]), )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

In [ ]:
@decorator
def filter_boring( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    stop_words = {'__dir__', '__str__','__repr__','__doc__','__hash__'}
    yield from filter( lambda e:e[1] not in stop_words, wrapped( _edge, _trav ) )

In [ ]:
@adapter_graphviz
@filter_boring
def all_attr(var, _):
    for attr_name, _ in zip(dir(var), range(50)):
        try:
            attr = getattr(var, attr_name)
            if hasattr( attr, "__call__"):
                # try:
                    attr = attr()
                # except Exception as ex:
                #     attr = str(ex)[:50]
            if attr is not None: #hasattr( attr, "__hash__"):
                yield (str(attr), str(attr_name))
        except:
            continue

trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=all_attr)
trav_forward.start_from( 10, build_paths=True )
for _, _ in zip(trav_forward, range(50)):
    pass

In [ ]:
all_attr.dot              # pylint: disable=pointless-statement

In [ ]:
all_attr.dot.attr( layout="neato")
all_attr.dot              # pylint: disable=pointless-statement

In [ ]:
('10',).__getnewargs__()